In [80]:
import pyaudio
import sys
import audioop
import struct
import numpy as np
import matplotlib.pyplot as plt
import wave
from scipy.signal import argrelextrema
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [88]:
class TapDetector():
    def __init__(this,channels,rate,index,frames,chunk,tapSize):
        this.audio = pyaudio.PyAudio()
        this.fullFrames = np.full((tapSize,chunk),None,dtype='float64')
        this.frames = np.zeros(tapSize)
        this.all = []
        this.tapSize = tapSize
        this.currentFrame = 0
        this.chunk = chunk
        this.rate = rate
        this.cooldown =0
        this.trainingSets = []
        this.negativeTrainingSet = []
        this.clf = None
        this.classifier = KNeighborsClassifier(2)#DecisionTreeClassifier(max_depth=10)#KNeighborsClassifier(1)
        this.stream = this.audio.open(format = pyaudio.paInt16,
                                      channels = channels,
                                      rate = rate,
                                      input = True,
                                      input_device_index = index,
                                      frames_per_buffer = frames)
        #FILE READING
        this.wavFile = None
    
    def start(this,seconds):
        print("Recording Started")
        print(not this.clf == None)
        for i in range(0,int(this.rate / this.chunk * seconds)):
            if this.record():
                if not this.clf == None:
                    flat = np.array([np.array(x).flatten() for x in this.fullFrames]).flatten()
                    prediction = this.clf.predict([flat])
                    print(prediction)
        print("Recording Finnished")
    
    def save(this,fileName):
        waveFile = wave.open(fileName, 'wb')
        waveFile.setnchannels(1)
        waveFile.setsampwidth(this.audio.get_sample_size(pyaudio.paInt16))
        waveFile.setframerate(this.rate)
        waveFile.writeframes(b''.join(this.all))
        waveFile.close()
    
    def read(this):
        if this.wavFile == None:
            data =this.stream.read(this.chunk, exception_on_overflow=False)
            return data
        else:
            return this.wavFile.readframes(this.chunk)
            
    
    def detectFromFile(this,fileName):
        this.wavFile = wave.open(fileName, 'rb')
        this.stream = this.audio.open(format =
                                            this.audio.get_format_from_width(this.wavFile.getsampwidth()),
                                            channels = this.wavFile.getnchannels(),
                                            rate = this.wavFile.getframerate(),
                                            output = True)
    def detectFromMic(this,channels,index,frames):
        this.wavFile = None
        this.stream = this.audio.open(format = pyaudio.paInt16,
                                      channels = channels,
                                      rate = this.rate,
                                      input = True,
                                      input_device_index = index,
                                      frames_per_buffer = frames)
        
    def getFrequency(this,data):
        return data
        fft = np.abs(np.fft.fft(data).real)
        fft = fft[:int(len(fft)/2)]
        freq = np.fft.fftfreq(this.chunk,1./this.rate)
        freq = freq[:int(len(freq)/2)]
        #print(fft[np.where(freq>0)[0][0]],20*np.log10(np.abs(np.fft.rfft(data))))
        return np.abs(np.fft.rfft(data))
        return 20*np.log10(np.abs(np.fft.rfft(data)))
    
    def record(this):
        tapFound = False
        data = this.read()
        if(len(data)<1):
            return tapFound
        decoded = np.frombuffer(data, dtype='<i2')
        this.all.append(data)
        this.frames[this.currentFrame] = audioop.rms(data, 2)
        this.fullFrames[this.currentFrame] = this.getFrequency(decoded)
        this.currentFrame = (this.currentFrame+1)%this.tapSize
        if(this.cooldown <= 0):
            tapFound = this.isTap(9000)
            return tapFound
        else:
            this.cooldown -= 1
        return tapFound
    
    def callibrate(this,buttons,tapsNeeded):
        for button in range(0,buttons):
            this.trainingSets.append([])
            print("\nCallibrating button position {}".format(button))
            for tap in range(1,tapsNeeded+1):
                while(not this.record()):
                    d = 1
                this.trainingSets[button].append(this.fullFrames.copy())
                sys.stdout.write("\r{}/{} taps registered for button {}".format(tap,tapsNeeded,button))
                sys.stdout.flush()
        print("False Positives taps:")
        for tap in range(1,tapsNeeded+1):
            while(not this.record()):
                d = 1
            this.negativeTrainingSet.append(this.fullFrames.copy())
            sys.stdout.write("\r{}/{} false taps registered so far".format(tap,tapsNeeded))
            sys.stdout.flush()
        this.buttonTraining()

    def buttonTraining(this):
        positiveTrainX, positiveTrainY = [], np.array([])
        for button in range(1,len(this.trainingSets)+1):
            positiveTrainX.extend([np.array(x).flatten() for x in this.trainingSets[button-1]])
            positiveTrainY = np.hstack([positiveTrainY,np.full( len(this.trainingSets[button-1]), button)])
        negativeTrainX = [np.array(x).flatten() for x in this.negativeTrainingSet]
        negativeTrainY = np.full( len(negativeTrainX), -1)
        trainX = positiveTrainX
        trainY = positiveTrainY
        print(np.array(trainX).shape,trainY.shape)
#        trainX.extend(negativeTrainX)
        #trainY = np.hstack([positiveTrainY,negativeTrainY])
        this.clf = this.classifier.fit(trainX,trainY)
        print(this.clf.predict(trainX))
        
    def stop(this):
        this.stream.stop_stream()
        this.stream.close()
        this.audio.terminate()
    
    def plot(this):
        plt.plot(np.hstack([this.frames[this.currentFrame:],this.frames[:this.currentFrame]]))
        print(this.fullFrames)
        plt.show()
    
    def isTap(this,threshold):
        soundSlice = np.hstack([this.frames[this.currentFrame:],this.frames[:this.currentFrame]])
        freqSlice = np.vstack([this.fullFrames[this.currentFrame:],this.fullFrames[:this.currentFrame]])
        diff = np.diff(soundSlice)
        maxPeaks = argrelextrema(soundSlice, np.greater)
        if len(maxPeaks) > 0:
            maxPeaks = maxPeaks[0].tolist()
        if len(maxPeaks) == 0:
            return False
        if soundSlice[maxPeaks[0]] > threshold and diff[maxPeaks[0]]**2 > 5000 and maxPeaks[0] > 1:
            for i in range(1,len(maxPeaks)):
                if soundSlice[maxPeaks[i]] > (soundSlice[maxPeaks[0]] * 0.5):
                    return False
            this.cooldown = 50//2
            return True
        return False
                   
    

In [ ]:
channels =1
rate = 44100
index = 0
frames = 5
chunk = 1024
tapSize = 7
detector = TapDetector(channels,rate,index,frames,chunk,tapSize)
detector.detectFromFile("test4buttons50training.wav")
detector.callibrate(4,50)
#detector.save("x.wav")
detector.detectFromMic(channels,index,frames)
detector.start(10)
detector.plot()
detector.stop()


Callibrating button position 0
50/50 taps registered for button 0
Callibrating button position 1
43/50 taps registered for button 1

In [ ]:
        
    def findPeaks(this,soundSlice,threshold):
        diff = np.diff(soundSlice)
        startPeak,startId = 0,0
        lowPeak,lowId = 0,0
        afterPeak, afterId = 0,0
        index = 0
        for db in soundSLice:
            index += 1
            if(lowId = 0 and db > 0 and db > startPeak and db >= threshold):
                startPeak = db
                startId = index
            elif(afterId = 0 and db < 0 and db < lowPeak):
                lowPeak = db,
                lowId = index
            elif(index > lowId and db > 0 and db > afterPeak):
                afterPeak = db
                afterId = index
            elif(after)